In [1]:
from collections import defaultdict

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from ydata_profiling import ProfileReport

from torch.utils.data import Dataset, DataLoader
from torch import nn
import torch

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from torchmetrics import MeanSquaredError, MeanAbsoluteError

import optuna

from pytorch_tools import CreateDataset, train_model_cls, train_model_reg, plot_metrics
from train_models import RegressorTrainer

/opt/anaconda3/envs/goit/lib/python3.10/site-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.6' (you have '2.0.5'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
df = pd.read_csv('data/ConcreteStrengthData.csv')

In [3]:
df

,CementComponent,BlastFurnaceSlag,FlyAshComponent,WaterComponent,SuperplasticizerComponent,CoarseAggregateComponent,FineAggregateComponent,AgeInDays,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30
...,...,...,...,...,...,...,...,...,...
1025,276.4,116.0,90.3,179.6,8.9,870.1,768.3,28,44.28
1026,322.2,0.0,115.6,196.0,10.4,817.9,813.4,28,31.18
1027,148.5,139.4,108.6,192.7,6.1,892.4,780.0,28,23.70
1028,159.1,186.7,0.0,175.6,11.3,989.6,788.9,28,32.77


In [4]:
X = df.drop(columns=['Strength'])
y = df['Strength']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((824, 8), (206, 8), (824,), (206,))

In [5]:
train_dataset = CreateDataset(X_train, y_train)
test_dataset = CreateDataset(X_test, y_test)

train_dataloader = DataLoader(train_dataset,
                              batch_size=40,
                              num_workers=0
                             )

test_dataloader = DataLoader(test_dataset,
                              batch_size=40,
                              num_workers=0
                             )

In [6]:
def objective(trial):
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True)
    hidden1 = trial.suggest_int("hidden1", 64, 512)
    hidden2 = trial.suggest_int("hidden2", 32, hidden1)
    hidden3 = trial.suggest_int("hidden3", 16, hidden2)

    class LinearModel(nn.Module):
        def __init__(self, in_dim, out_dim=1):
            super().__init__()
            self.features = nn.Sequential(
                nn.Linear(in_dim, hidden1),
                nn.ReLU(),
                nn.Linear(hidden1, hidden2),
                nn.ReLU(),
                nn.Linear(hidden2, hidden3),
                nn.ReLU(),
                nn.Linear(hidden3, out_dim),
            )
        def forward(self, x):
            return self.features(x)

    model = LinearModel(in_dim=X_train.shape[1], out_dim=1)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()

    trainer = RegressorTrainer(model=model,
                                criterion=criterion,
                                optimizer=optimizer,
                                device='cpu')

    trainer.fit(train_dataloader,
                test_dataloader,
                num_epoch=20,
                info_every_iter=20,
                show_val_metrics=False)

    return trainer.metrics['train_loss'][-1]

In [7]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=5)

[I 2025-05-02 12:03:32,033] A new study created in memory with name: no-name-07272458-0b41-48f2-95b0-33f03ca88bb6


  0%|          | 0/20 [00:00<?, ?it/s]

[I 2025-05-02 12:03:32,663] Trial 0 finished with value: 130.6831765035981 and parameters: {'lr': 0.0009118694879138537, 'hidden1': 112, 'hidden2': 43, 'hidden3': 29}. Best is trial 0 with value: 130.6831765035981.


Epoch [20/20] Train Loss: 130.6832 MSE: 130.6832 MAE: 9.2615 RMSE: 11.4317


  0%|          | 0/20 [00:00<?, ?it/s]

...

[I 2025-05-02 12:03:35,318] Trial 3 finished with value: 176.61419418483104 and parameters: {'lr': 0.0001790092067401318, 'hidden1': 310, 'hidden2': 192, 'hidden3': 101}. Best is trial 1 with value: 29.34816534542343.


Epoch [20/20] Train Loss: 176.6142 MSE: 176.6142 MAE: 10.7952 RMSE: 13.2896


  0%|          | 0/20 [00:00<?, ?it/s]

[I 2025-05-02 12:03:36,033] Trial 4 finished with value: 26.537802224020357 and parameters: {'lr': 0.005879062985942876, 'hidden1': 305, 'hidden2': 155, 'hidden3': 78}. Best is trial 4 with value: 26.537802224020357.


Epoch [20/20] Train Loss: 26.5378 MSE: 26.5378 MAE: 3.9355 RMSE: 5.1515


In [8]:
print("Best trial:")
print(study.best_trial.params)

Best trial:
{'lr': 0.005879062985942876, 'hidden1': 305, 'hidden2': 155, 'hidden3': 78}


In [9]:
class Model(torch.nn.Module):
    def __init__(self, in_dim, out_dim=1):
        super().__init__()
        
        self.features = torch.nn.Sequential(
            nn.Linear(in_dim, 144),
            torch.nn.ReLU(),
            
            nn.Linear(144, 72),
            torch.nn.ReLU(),
            
            nn.Linear(72, 72),
            torch.nn.ReLU(),
            
            nn.Linear(72, out_dim),
        )
    
        
    def forward(self, x):
        output = self.features(x)
        return output
    

model = Model(in_dim=X_train.shape[1], out_dim=1)
  
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.002)

metrics, model = train_model_reg(num_epoch=20,
                    train_dataloader = train_dataloader,
                    test_dataloader = test_dataloader,
                    model=model,
                    criterion=criterion,
                    optimizer=optimizer,
                    )

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch [1/20] Train Loss: 1359.0050 MSE: 1359.0049 MAE: 32.7752 RMSE: 36.8647
Epoch [2/20] Train Loss: 369.6757 MSE: 369.6756 MAE: 14.9487 RMSE: 19.2270
Epoch [3/20] Train Loss: 221.4427 MSE: 221.4427 MAE: 12.0597 RMSE: 14.8810
Epoch [4/20] Train Loss: 198.4956 MSE: 198.4957 MAE: 11.3621 RMSE: 14.0888
Epoch [5/20] Train Loss: 169.7581 MSE: 169.7581 MAE: 10.6264 RMSE: 13.0291
Epoch [6/20] Train Loss: 155.4281 MSE: 155.4281 MAE: 10.1588 RMSE: 12.4671
Epoch [7/20] Train Loss: 144.5332 MSE: 144.5332 MAE: 9.7783 RMSE: 12.0222
Epoch [8/20] Train Loss: 133.3538 MSE: 133.3538 MAE: 9.3684 RMSE: 11.5479
Epoch [9/20] Train Loss: 121.3408 MSE: 121.3408 MAE: 8.8917 RMSE: 11.0155
Epoch [10/20] Train Loss: 108.2449 MSE: 108.2449 MAE: 8.3407 RMSE: 10.4041
Epoch [11/20] Train Loss: 94.3646 MSE: 94.3646 MAE: 7.7023 RMSE: 9.7141
Epoch [12/20] Train Loss: 80.6079 MSE: 80.6078 MAE: 7.0275 RMSE: 8.9782
Epoch [13/20] Train Loss: 68.4859 MSE: 68.4858 MAE: 6.4036 RMSE: 8.2756
Epoch [14/20] Train Loss: 59.3470 M

In [10]:
y_true = []
y_pred = []

with torch.no_grad():
    for X_batch, y_batch in test_dataloader:
        predictions = model(X_batch).squeeze()
        
        y_true.extend(y_batch.numpy())
        y_pred.extend(predictions.numpy())

r2 = r2_score(y_true, y_pred)
mse = mean_squared_error(y_true, y_pred)
mae = mean_absolute_error(y_true, y_pred)

print(f"R²: {r2:.4f}")
print(f"MSE: {mse:.4f}")
print(f"MAE: {mae:.4f}")

R²: 0.8130
MSE: 48.1893
MAE: 5.4075
